<a href="https://colab.research.google.com/github/SK-0911/Image-Editing-App/blob/main/Image_Editing_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installations

In [ ]:
!pip install streamlit==1.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.086s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


Haar cascade files are pre-trained detectors of some parts of the body, such as faces, mouths, eyes, etc.

In [ ]:
%%writefile app.py
import streamlit as st  # For creating the webapp
import cv2
from PIL import Image, ImageEnhance
import numpy as np
import os

face_cascade = cv2.CascadeClassifier("/content/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("/content/haarcascade_eye.xml")

def detect_faces(face_img):
  new_img = np.array(face_img.convert("RGB"))
  faces = face_cascade



def apply_vignette(img_arr, strength=0.5):
    rows, cols, _ = img_arr.shape
    y, x = np.ogrid[:rows, :cols]
    center = (rows / 2, cols / 2)
    mask = np.exp(-((x - center[1]) ** 2 + (y - center[0]) ** 2) / (2 * (strength * center[1]) ** 2))

    # Normalize the mask values to be in the range [0.0, 1.0]
    mask = (mask - np.min(mask)) / (np.max(mask) - np.min(mask))

    # Apply the vignette effect
    vignetted_img = img_arr * mask[:, :, np.newaxis]

    return np.clip(vignetted_img, 0, 255).astype(np.uint8)


def main():
  st.title("Image Editing App")
  st.text("Edit your Images in a fast and simple way")

  activities = ["Detection", "About"]
  choice = st.sidebar.selectbox("Select Activity", activities)

  if choice == "Detection":
    st.subheader("Face Detection")
    img_file = st.file_uploader("Upload Image", type=["jpg", "png", "jpeg"])

    if img_file is not None:
      img = Image.open(img_file)
      st.text("Original Image")
      st.image(img) # Displaying the image

      enhance_type = st.sidebar.radio("Enhance type", ["Original", "Grey-scale",
                                                       "Contrast", "Brightness",
                                                       "Blur", "Sharpness",
                                                       "Saturation", "Sepia Tone",
                                                       "Edge Enhancement", "Histogram Equalization",
                                                       "Vignette Effect", "Hue Adjustment"])

      if enhance_type == "Grey-scale":
        img_arr = np.array(img.convert('RGB'))
        gray = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
        st.image(gray)

      elif enhance_type == "Contrast":
        rate = st.sidebar.slider("Contrast Level", 0.1, 5.0)
        enhancer = ImageEnhance.Contrast(img)
        enhanced_img = enhancer.enhance(rate)
        st.image(enhanced_img)

      elif enhance_type == "Brightness":
        rate = st.sidebar.slider("Brightness", 0.0, 10.0)
        brightness = ImageEnhance.Brightness(img)
        brighter_img = brightness.enhance(rate)
        st.image(brighter_img)

      elif enhance_type == "Blur":
        rate = st.sidebar.slider("Blurriness", 0.0, 10.0)
        blur_img = cv2.GaussianBlur(np.array(img), (15,15), rate)
        st.image(blur_img)

      elif enhance_type == "Sharpness":
        rate = st.sidebar.slider("Sharpness Level", 0.0, 10.0)
        sharp = ImageEnhance.Sharpness(img)
        sharp_img = sharp.enhance(rate)
        st.image(sharp_img)

      elif enhance_type == "Saturation":
        saturation_rate = st.sidebar.slider("Saturation Level", 0.0, 5.0)
        saturation = ImageEnhance.Color(img)
        saturated_img = saturation.enhance(saturation_rate)
        st.image(saturated_img)

      elif enhance_type == "Sepia Tone":
        img_arr = np.array(img.convert('RGB'))

        # Define Sepia Tone matrix
        sepia_matrix = np.array([[0.393, 0.769, 0.189],
                                [0.349, 0.686, 0.168],
                                [0.272, 0.534, 0.131]])

        # Add a slider for Sepia Tone intensity
        sepia_intensity = st.sidebar.slider("Sepia Intensity", 0.0, 1.0, 0.5)

        sepia_img = np.dot(img_arr, sepia_matrix.T)
        sepia_img = np.clip(sepia_img * sepia_intensity, 0, 255).astype(np.uint8)

        st.image(sepia_img)

      elif enhance_type == "Edge Enhancement":
        img_arr = np.array(img.convert('RGB'))
        gray = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)

        edges = cv2.Sobel(gray, cv2.CV_64F, 1, 1, ksize=3)
        enhanced_img = cv2.convertScaleAbs(edges)
        st.image(enhanced_img, channels="GRAY")

      elif enhance_type == "Histogram Equalization":
        img_arr = np.array(img.convert('RGB'))
        gray = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)

        # Apply Histogram Equalization
        equalized_img = cv2.equalizeHist(gray)

        st.image(equalized_img, channels="GRAY")

      elif enhance_type == "Vignette Effect":
        img_arr = np.array(img.convert('RGB'))

        # Add a slider for Vignette effect strength
        vignette_strength = st.sidebar.slider("Vignette Strength", 0.1, 1.0, 0.5)

        # Apply Vignette Effect
        vignetted_img = apply_vignette(img_arr, strength=vignette_strength)

        st.image(vignetted_img)

      elif enhance_type == "Hue Adjustment":
        img_arr = np.array(img.convert('RGB'))

        # Add a slider for Hue Adjustment
        hue_factor = st.sidebar.slider("Hue Factor", -1.0, 1.0, 0.0)

        # Apply Hue Adjustment
        img_hsv = cv2.cvtColor(img_arr, cv2.COLOR_RGB2HSV)
        h, s, v = cv2.split(img_hsv)

        h = np.mod((h + int(hue_factor * 360)), 180).astype(np.uint8)

        img_hsv = cv2.merge((h, s, v))
        hue_adjusted_img = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)

        st.image(hue_adjusted_img)

      elif enhance_type == "Original":
        st.image(img, width=300)

      else:
        st.image(img, width=300)

    tasks = ["Faces", "Eyes", "Cartoonize", "Cannize"]
    feature_choice = st.sidebar.selectbox("Find features", tasks)
    if st.button("Process Image"):
      if feature_choice == "Face":
        result_img, result_face = detect_faces(img)
        st.image(result_img)



  elif choice == "About":
    st.subheader("About the Developer")
    st.markdown("Built with ❤️ by [Shubham Kumar](https://www.linkedin.com/in/shubham-911/)")
    st.text("My name is Shubham Kumar I am a Computer Science Student at NMIMS Navi Mumbai")


main()

Writing app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!curl https://loca.lt/mytunnelpassword

34.106.217.30

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 3.221s
your url is: https://fresh-coins-draw.loca.lt
